In [ ]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, TimeDistributed, multiply, Lambda, InputLayer
from tensorflow.keras.layers import LSTM, GRU, LeakyReLU, BatchNormalization, ReLU, concatenate, Masking, Permute, Reshape
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, RepeatVector,Bidirectional, GlobalAveragePooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam, SGD, Nadam
from tensorflow.keras.initializers import *
from tensorflow.keras import Model
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

np.random.seed(73) 


def getmodel(n1, dp1, ac, opt, los, learn):
  mcdp = True
  
  if ac == 1:
    ac = 'relu'
  elif ac ==2:
    ac = 'selu'
  else: 
    ac = 'elu'

  if los ==1:
    loss = 'mse'
  elif los ==2:
    loss = 'mae'
  else: 
    loss = 'huber'
  
  if opt == 1:
    opt = Adam(lr=learn)
  elif opt == 2:
    opt = Nadam(lr=learn)
  else: 
    opt = RMSprop(lr = learn)
  input = Input(shape=(14,))
  dense = Dense(n1, activation=ac)(input)
  dense = Dropout(dp1)(dense, training=mcdp)
  dense = Dense(n1,activation=ac)(dense)
  dense = Dropout(dp1)(dense, training=mcdp)
  dense = Dense(n1,activation=ac)(dense)
  dense = Dropout(dp1)(dense, training=mcdp)
  output1 = Dense(1, activation='linear')(dense)

  model = Model(inputs= input, outputs=output1)
  #plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

  #model.summary()
  print(dp1, ac, loss, opt, learn, n1)
  model.compile(optimizer=opt, loss=loss)
  return model 


In [ ]:
def remove_outlier(df_in, col_name):
  q1 = df_in[col_name].quantile(0.15)
  q3 = df_in[col_name].quantile(0.85)
  iqr = q3-q1 #Interquantile range
  fence_low  = q1-1.5*iqr
  fence_high = q3+1.5*iqr
  df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
  return df_out

df = pd.read_csv('ropaxdata.csv', sep=',')
namarr = ['target','fuelDensity.csv','fuelTemp.csv','inclinometer-raw.csv', 'longitudinalWaterSpeed.csv','level1median.csv','level2median.csv', 'portPitch.csv', 'portRudder.csv','speedKnots.csv','starboardPitch.csv', 'starboardRudder.csv','trueHeading.csv', 'windAngle.csv', 'windSpeed.csv']
df.dropna(inplace=True)
df = df[namarr]
#print(df.isnull().sum())

for nam in namarr: 
  df = remove_outlier(df,nam)
print(df.shape)
df = df.sample(frac=0.5, replace=False, random_state=42, axis=0)
#sns.pairplot(data = df, vars=namarr)
#plt.show()
bounds = [{'name': 'l1',                'type': 'discrete',    'domain': (50,75,100)},
          {'name': 'l1_drop',          'type': 'continuous',  'domain': (0.1, 0.3)},
          {'name': 'activation',                'type': 'discrete',    'domain': (1,2,3)},
          {'name': 'optimizer',                'type': 'discrete',    'domain': (1,2,3)},
          {'name': 'loss',                'type': 'discrete',    'domain': (1,2)},
          {'name': 'learn',       'type': 'discrete',    'domain': (1e-2,1e-3,1e-4)},
          {'name': 'batchsize',                'type': 'discrete',    'domain': (32,64,128)},
          {'name': 'epochs',                'type': 'discrete',    'domain': (25,38,50)}]          

(1220307, 15)


In [ ]:
import GPy, GPyOpt
import tqdm
import sklearn
from sklearn import model_selection
X = df.drop(['target'], axis=1).to_numpy()
Y = df['target'].to_numpy()
#X = sklearn.preprocessing.StandardScaler().fit_transform(X)
X = sklearn.preprocessing.MinMaxScaler().fit_transform(X)
#Y = np.divide(np.subtract(Y, Y.mean()), Y.std())
Y = np.divide(np.subtract(Y, np.min(Y)), np.max(Y)-np.min(Y))
Xtest, Xpool, ytest, ypool = sklearn.model_selection.train_test_split(X,Y,test_size=0.7, shuffle=True)
Xptrain, Xptest, yptrain, yptest = model_selection.train_test_split(X,Y,test_size=0.3, shuffle=True)

print(Xptrain.shape, yptrain.shape)

##def getmodel(hidden, n1, n2,dp1, dp2, mcdp,bs, ep,  ac, opt, learn, loss)

def passivelearn(x): 
  model = getmodel(int(x[:,0]),  float(x[:,1]),  int(x[:,2]), int(x[:,3]),  int(x[:,4]),  float(x[:,5]))
  #def getmodel(n1, n2,dp1, dp2, mcdp, op, learn):
  history = model.fit(Xptrain,yptrain,batch_size=int(x[:,6]),  epochs=int(x[:,7]), verbose=0) 
  r2_arr = []
  for i in tqdm.tqdm(range(50)):
    yhat = model.predict(Xptest, batch_size=int(x[:,6]))
    r2_arr.append(sklearn.metrics.r2_score(yptest,yhat))
  eval = np.mean(r2_arr)
  print(eval)
  return eval


opt = GPyOpt.methods.BayesianOptimization(f=passivelearn, model_type='GP', domain=bounds, maximize=True)

# optimize mnist model
opt.run_optimization(max_iter=10)
print(opt.x_opt, opt.fx_opt)

opt.plot_acquisition()


(427107, 14) (427107,)
0.1946947820699229 selu mae <tensorflow.python.keras.optimizer_v2.rmsprop.RMSprop object at 0x7ff089426ba8> 0.001 100


100%|██████████| 50/50 [03:30<00:00,  4.21s/it]


0.6660766326132693
0.19441915236627683 selu mae <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7ff083ce9748> 0.0001 75


100%|██████████| 50/50 [01:48<00:00,  2.16s/it]


0.7283985360282272
0.26447286560528893 relu mae <tensorflow.python.keras.optimizer_v2.nadam.Nadam object at 0x7ff08addd240> 0.01 75


...

0.7491233877356358
0.1 relu mse <tensorflow.python.keras.optimizer_v2.rmsprop.RMSprop object at 0x7ff08285e550> 0.001 50


100%|██████████| 50/50 [00:55<00:00,  1.12s/it]


0.7479587350515944
0.1 relu mse <tensorflow.python.keras.optimizer_v2.rmsprop.RMSprop object at 0x7ff08935c0b8> 0.001 50


100%|██████████| 50/50 [00:55<00:00,  1.12s/it]


0.740268800528561
[5.00e+01 1.00e-01 1.00e+00 3.00e+00 1.00e+00 1.00e-03 1.28e+02 3.80e+01] -0.7566675646738646
